# 📶 Telecomunicaciones: churn, ARPU y calidad de red
KPIs: churn, ARPU, NPS, disponibilidad, fallas por sitio.

## Casos de uso
- Predicción de churn y ofertas de retención
- Optimización de celdas y capacidad
- Detección de fraude en recargas y SIM swapping
- SLA por cliente y reporte regulatorio

In [1]:
# Mini cálculo ARPU
ingresos = [10, 12, 8, 15]
subs = 4
sum(ingresos)/max(subs,1)

11.25

## Arquitectura
- CDRs en streaming, enriquecimiento con GIS
- Medición QoE/QoS y SLA por segmento
- Feature store + modelos de churn y capacity planning

## 💼 Valor en Telco
- Menor churn con retención proactiva y ofertas personalizadas.
- ARPU ↑ mediante upsell/cross-sell y optimización de capacidad.
- SLA y QoE transparentes por segmento/cliente.


In [2]:
# Práctica: propensión de churn (toy)
import pandas as pd
import math

df = pd.DataFrame({
    'tenure_meses':[1,3,6,12,24,36],
    'uso_gb':[1.2, 3.5, 5.0, 7.1, 10.3, 12.5],
    'reclamos_90d':[2,1,1,0,0,0]
})
# score simple: más reclamos y poco tenure → mayor churn
z = -1.5 + 0.1*df['reclamos_90d'] - 0.02*df['tenure_meses'] - 0.01*df['uso_gb']
df['churn_prob'] = 1/(1+(-z).apply(math.exp))
print(df)


   tenure_meses  uso_gb  reclamos_90d  churn_prob
0             1     1.2             2    0.208829
1             3     3.5             1    0.183172
2             6     5.0             1    0.172216
3            12     7.1             0    0.140517
4            24    10.3             0    0.110760
5            36    12.5             0    0.087464


## 🏗️ Arquitectura CDR/GIS/SLA
- CDRs en streaming + enriquecimiento GIS → medición QoE y congestión.
- Feature store para churn y capacity planning.
- APIs de SLA por cliente y reportes regulatorios.


In [3]:
# Visual: churn por cliente (toy)
import plotly.express as px
fig = px.bar(df.reset_index().rename(columns={'index':'id'}), x='id', y='churn_prob', title='Probabilidad de churn por registro')
fig.show()


## ✅ Checklist de estrategia de negocio y datos
- Objetivo (churn↓, ARPU↑, SLA/QoE): …
- KPIs: churn, ARPU, NPS, SLA/QoS.
- Palancas: CDR+GIS, modelos churn, capacity planning.
- Datos: CDRs, reclamos, uso; calidad/latencia.
- Gobierno: dominios, reporte regulatorio.
- SLOs: latencia scoring, disponibilidad, costo.
- ROI/supuestos: …
- Riesgos: sesgos, privacidad, fraudes.


## 🔗 Puente Estrategia ↔ Ingeniería de Datos
- OKR: churn mensual < 1.5% y ARPU ↑ → KPI negocio: churn, ARPU → KPI datos: latencia CDR, QoE/QoS → Capacidades: streaming CDR + GIS + feature store → Decisión: retención y capacity planning → Impacto: Δ ingresos.

```
[CDRs]→[Streaming+GIS]→[Features churn]→[Score/Segmentos]→[Ofertas]→[Retención]→[Δ ARPU]
```


In [4]:
# contrato_de_datos (telco)
contrato_datos = {
    "objetivo_negocio": "Churn < 1.5%",
    "kpi_negocio": "Tasa de churn",
    "kpi_datos": {"latencia_cdr_s": "<60", "cobertura_qoe_qos": ">=98%"},
    "dataset": ["cdrs", "clientes", "red"],
    "propietario": "Dominio Clientes/Red",
    "slo": {"frescura_features_min": "<=30"},
    "decision": "Ofertas retención y capacity planning",
    "impacto_economico_usd": 200000,
}
print(contrato_datos)


{'objetivo_negocio': 'Churn < 1.5%', 'kpi_negocio': 'Tasa de churn', 'kpi_datos': {'latencia_cdr_s': '<60', 'cobertura_qoe_qos': '>=98%'}, 'dataset': ['cdrs', 'clientes', 'red'], 'propietario': 'Dominio Clientes/Red', 'slo': {'frescura_features_min': '<=30'}, 'decision': 'Ofertas retención y capacity planning', 'impacto_economico_usd': 200000}


## 📋 Caso de uso: Retención de clientes con feature store

**Contexto de negocio:** Un operador telco pierde 2.5% de clientes/mes (churn). Cada cliente perdido representa $850 de LTV. Objetivo: churn <1.5%.

**Problema sin ingeniería de datos:**
- Modelo de churn entrenado con datos de hace 3 meses.
- Features calculadas manualmente en batch diario (lag de 24h).
- Sin capacidad de ofrecer retención proactiva.

**Solución con ingeniería de datos:**
1. Pipeline streaming: CDRs en tiempo real + enriquecimiento geográfico (GIS).
2. Feature store: 80+ features pre-calculadas (uso, reclamos, QoE, tenure) con refresh cada 15 min.
3. Modelo de propensión a churn actualizado semanalmente con datos frescos.
4. API de retención: detecta clientes en riesgo y dispara oferta automática.
5. A/B testing embebido: mide efectividad de cada estrategia de retención.

**Resultado:**
- Churn: 2.5% → 1.4% mensual.
- Tiempo de reacción: 24h → 15 min.
- ROI de campañas de retención: 3.2x (vs. 1.8x anterior).
- Ahorro anual: $6.8M en LTV recuperado.


In [5]:
# Ejercicio: pipeline de features de churn
import pandas as pd

# Simular datos de clientes
clientes = pd.DataFrame({
    'cliente_id': range(1, 11),
    'tenure_meses': [2, 12, 24, 6, 36, 3, 18, 1, 48, 9],
    'uso_gb_mes': [1.5, 8.2, 12.0, 3.5, 15.0, 0.8, 10.5, 0.5, 18.0, 5.2],
    'reclamos_90d': [3, 0, 0, 2, 0, 4, 1, 5, 0, 1],
    'pago_promedio': [25, 45, 60, 30, 70, 20, 50, 18, 80, 35],
})

# Feature engineering (simulando feature store)
clientes['riesgo_bajo_uso'] = (clientes['uso_gb_mes'] < 2).astype(int)
clientes['riesgo_reclamos'] = (clientes['reclamos_90d'] > 2).astype(int)
clientes['riesgo_nuevo'] = (clientes['tenure_meses'] < 6).astype(int)
clientes['churn_score'] = (
    clientes['riesgo_bajo_uso'] * 30 +
    clientes['riesgo_reclamos'] * 40 +
    clientes['riesgo_nuevo'] * 30
)

alto_riesgo = clientes[clientes['churn_score'] >= 60]
print(f"Clientes en riesgo alto de churn: {len(alto_riesgo)}")
print(alto_riesgo[['cliente_id', 'tenure_meses', 'reclamos_90d', 'churn_score']])
print("\n💡 Feature store permite calcular esto cada 15 min → oferta proactiva")


Clientes en riesgo alto de churn: 3
   cliente_id  tenure_meses  reclamos_90d  churn_score
0           1             2             3          100
5           6             3             4          100
7           8             1             5          100

💡 Feature store permite calcular esto cada 15 min → oferta proactiva


---

## 🧭 Navegación

**← Anterior:** [⚡ Energía y Recursos Naturales: continuidad, costos y seguridad](05_energia_recursos_naturales.ipynb)

**Siguiente →:** [🏭 Industria y Manufactura: OEE, calidad y throughput →](07_industria_manufactura.ipynb)

**📚 Índice de Negocio LATAM:**
- [🎯 Estrategia de Datos en LATAM: de la visión al ROI](01_estrategia_datos_latam.ipynb)
- [🛒 Retail y Consumo Masivo: demanda, surtido y pérdidas](02_retail_consumo_masivo.ipynb)
- [💳 Finanzas y Banca: riesgo, fraude, rentabilidad](03_finanzas_banca.ipynb)
- [🏥 Salud y Farmacéutico: calidad, trazabilidad, acceso](04_salud_farmaceutico.ipynb)
- [⚡ Energía y Recursos Naturales: continuidad, costos y seguridad](05_energia_recursos_naturales.ipynb)
- [📶 Telecomunicaciones: churn, ARPU y calidad de red](06_telecomunicaciones.ipynb) ← 🔵 Estás aquí
- [🏭 Industria y Manufactura: OEE, calidad y throughput](07_industria_manufactura.ipynb)
- [🚚 Logística y Transporte: costo por entrega, SLA, ocupación](08_logistica_transporte.ipynb)
- [🌾 Agro y Alimentos: rendimiento, calidad y trazabilidad](09_agro_alimentos.ipynb)
- [🏛️ Sector Público y Gobierno: transparencia, servicio y eficiencia](10_sector_publico_gobierno.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
